## Add automated charge identification to NOMAD OASIS

Enable that the nomad-parser-nexus apm dataconverter can automatically<br>
analyze the information for each defined range in a technology partner<br>
atom probe ranging definition files so that for each range we can find<br>
if it is possible to recover the charge information that is typically<br>
not stored in such files.<br>

We use a combinatorial analysis because of which the larger it is the<br>
number of atoms in the molecular ion the substantially longer it takes<br>
to perform the analysis for each ion. The reason is that all possible<br>
combinations of isotopes for the elements in the definition of the range<br>
is evaluated and checked for which charge it has and if its mass-to-charge<br>
is in the defined \[low, high\] interval.<br>

<div class="alert alert-block alert-danger">
We assume in NOMAD OASIS that isotope information is not ignored.<br>
With this we highlight the need for storing charge state information<br>
inside the ranging definitions file. NXion instances whose charge is<br>
0. means that it was impossible to uniquely recover the charge because<br>
there are multiple combinations of isotopes possible within the interval,<br>
i.e. the analysis of that range is underconstrained.<br>
</div>

## Load libs

In [1]:
import os
import numpy as np
import h5py
from jupyterlab_h5web import H5Web
from ifes_apt_tc_data_modeling.utils.utils import create_isotope_vector, isotope_vector_to_dict_keyword, isotope_vector_to_human_readable_name
from ifes_apt_tc_data_modeling.nexus.nx_ion import NxIon
from ifes_apt_tc_data_modeling.utils.molecular_ions import MolecularIonBuilder
from ifes_apt_tc_data_modeling.utils.utils import MAX_NUMBER_OF_ATOMS_PER_ION
from ifes_apt_tc_data_modeling.utils.nist_isotope_data import isotopes
from ase.data import atomic_numbers, atomic_masses, chemical_symbols

## Tests with NXion and low-level stuff

In [2]:
# ivec = create_isotope_vector(['K-40', 'Xe-126'])  # , 'U', 'H'])
# ivec = create_isotope_vector(['Tc', 'H', 'H'])  # 'C'])
# ivec = create_isotope_vector(['B', 'B', 'B', 'N', 'N', 'Ti'])  # 'N', 'N'])
# ivec = create_isotope_vector([])
ivec = create_isotope_vector(['O', 'O', 'O', 'O'])
ivec = create_isotope_vector(['Fe', 'Fe', 'O', 'O', 'O', 'O', 'O'])
# ivec = create_isotope_vector(['X'])
print(ivec)
# print(isotope_vector_to_dict_keyword(ivec))
# print(isotope_vector_to_human_readable_name(ivec, +4))

[26 26  8  8  8  8  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


## Take the information available in every range file.

In [3]:
ion = NxIon(isotope_vector=ivec, charge=0)
# ion.add_range((np.floor(atomic_masses[43]) - 2.)*0.5, (np.floor(atomic_masses[43]) + 4.)*0.5)
# ion.add_range(20.2840, 20.3680)
# ion.add_range(54.9540, 54.9630)
# ion.add_range(32.0050,  32.1370)
# ion.add_range(32.4280,  32.4910)
# ion.add_range(32.9600,  33.0640)
# ion.add_range(33.9720,  34.0380)
# ion.add_range(175.8060, 176.2460)
ion.add_range(95.6540, 96.1780)
# ion.add_range(10.5, 13.5)
# ion.add_range(13.0, 13.3)  # should not be prevented because this is equivalent to ion.add_range(10.0, 13.3)
ion.report()

ion_type

isotope_vector
[26 26  8  8  8  8  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
nuclid_list
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]
 [26 26  8  8  8  8  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0]]
human-readable name
Fe Fe O O O O O
charge
0
ranges
[[95.654 96.178]]
comment

color

volume



In [4]:
mion = MolecularIonBuilder(min_abundance=0.,
                           min_abundance_product=0.,
                           min_half_life=np.inf,
                           sacrifice_uniqueness=True,
                           verbose=True)
mion.combinatorics(
    ion.isotope_vector.typed_value,
    ion.ranges.typed_value[0, 0],
    ion.ranges.typed_value[0, 1])

MolecularIonBuilder initialized with 259
Maximum recursion depth 7
[26 26  8  8  8  8  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
Found 88 candidates!
Reduce set of 88 candidates to a unique...
Reduced set to 8 relevant candidates...
8218_7194_2056_2056_2056_2056_2056__2
7962_7194_2312_2056_2056_2056_2056__2
7706_7706_2056_2056_2056_2056_2056__2
7706_7194_2568_2056_2056_2056_2056__2
7706_7194_2312_2312_2056_2056_2056__2
7194_7194_2568_2568_2056_2056_2056__2
7194_7194_2568_2312_2312_2056_2056__2
7194_7194_2312_2312_2312_2312_2056__2
Multiple relevant candidates meet all selection criteria
Multiple relevant candidates have all the same charge 2


(2,
  <ifes_apt_tc_data_modeling.utils.molecular_ions.MolecularIonCandidate at 0x7f816ef85ed0>])

## RRNG

In [ ]:
from ifes_apt_tc_data_modeling.rrng.rrng_reader import ReadRrngFileFormat

In [ ]:
fnm = []

In [ ]:
# directory = os.getcwd()
# directory += '/rrng/examples_without_provenance/aut_leoben_reichmann/'
# print(directory)

In [ ]:
# for filename in os.listdir(directory):
for filename in fnm:
    if filename.lower().endswith(".rrng"):
        print(filename)
        if True is True:  # if filename in []:
            # a = ReadRrngFileFormat(directory + "/" + filename)
            a = ReadRrngFileFormat(filename)
            # a.rrng['molecular_ions']
            
            # h5w = h5py.File(directory + "/" + filename + ".nxs", 'w')
            h5w = h5py.File(filename + ".nxs", 'w')
            h5w.create_group("/entry")
            i = 1
            for ion in a.rrng['molecular_ions']:
                trg = "/entry/ion" + str(i)
                grp = h5w.create_group(trg)
                grp.attrs['NXclass'] = 'NXion'
                dst = h5w.create_dataset(trg + "/comment", data=ion.comment.typed_value)
                dst = h5w.create_dataset(trg + "/color", data=ion.color.typed_value)
                dst = h5w.create_dataset(trg + "/volume", dtype=np.float32,
                                         data=ion.volume.typed_value)
                dst.attrs['unit'] = 'nm^3'
                dst = h5w.create_dataset(trg + "/isotope_vector", dtype=np.uint16,
                                         data=ion.isotope_vector.typed_value, 
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(trg + "/nuclid_list", dtype=np.uint16,
                                         data=ion.nuclid_list.typed_value,
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(trg + "/charge", dtype=np.int8,
                                         data=ion.charge.typed_value)
                dst.attrs['unit'] = 'eV'
                dst = h5w.create_dataset(trg + "/name", data=ion.name.typed_value)
                # print(ion.name.typed_value)
                dst = h5w.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                                         data=ion.ranges.typed_value)
                dst.attrs['unit'] = 'Da'
                
                # charge model
                subgrpnm = trg + "/charge_model"
                subgrp = h5w.create_group(subgrpnm)
                subgrp.attrs['NXclass'] = "NXprocess"
                dst = h5w.create_dataset(subgrpnm + "/min_abundance", dtype=np.float64,
                                         data=ion.charge_model['min_abundance'])
                dst = h5w.create_dataset(subgrpnm + "/min_abundance_product", dtype=np.float64,
                                         data=ion.charge_model['min_abundance_product'])
                dst = h5w.create_dataset(subgrpnm + "/min_half_life", dtype=np.float64,
                                         data=ion.charge_model['min_half_life'])
                dst.attrs['unit'] = 's'
                dst = h5w.create_dataset(subgrpnm + "/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                                         data=ion.charge_model['sacrifice_isotopic_uniqueness'])
                dst = h5w.create_dataset(subgrpnm + "/isotope_matrix", dtype=np.uint16,
                                         data=ion.charge_model['isotope_matrix'],
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(subgrpnm + "/charge_vector", dtype=np.int8,
                                         data=ion.charge_model['charge_vector'])
                dst.attrs['unit'] = 'eV'
                dst = h5w.create_dataset(subgrpnm + "/mass_vector", dtype=np.float64,
                                         data=ion.charge_model['mass_vector'])
                dst.attrs['unit'] = 'Da'
                dst = h5w.create_dataset(subgrpnm + "/nat_abun_prod_vector", dtype=np.float64,
                                         data=ion.charge_model['nat_abun_prod_vector'])
                dst = h5w.create_dataset(subgrpnm + "/min_half_life_vector", dtype=np.float64,
                                         data=ion.charge_model['min_half_life_vector'])
                dst.attrs['unit'] = 's'
                       
                i += 1
            h5w.close()

In [ ]:
# H5Web(directory + "/" + "R23_Mass.nxs")
H5Web(filename)

## RNG

In [ ]:
from ifes_apt_tc_data_modeling.rng.rng_reader import ReadRngFileFormat

In [ ]:
fnm = []

In [ ]:
# directory = os.getcwd()
# directory += '/rng/examples_without_provenance/ger_duesseldorf_kuehbach/'
# print(directory)

In [ ]:
# for filename in os.listdir(directory):
for filename in fnm:
    if filename.lower().endswith(".rng"):
        print(filename)
        if True is True:  # if filename in []:
            # a = ReadRngFileFormat(directory + "/" + filename)
            a = ReadRngFileFormat(filename)
            # a.rng['molecular_ions']
            
            # h5w = h5py.File(directory + "/" + filename + ".nxs", 'w')
            h5w = h5py.File(filename + ".nxs", 'w')
            h5w.create_group("/entry")
            i = 1
            for ion in a.rng['molecular_ions']:
                trg = "/entry/ion" + str(i)
                grp = h5w.create_group(trg)
                grp.attrs['NXclass'] = 'NXion'
                dst = h5w.create_dataset(trg + "/comment", data=ion.comment.typed_value)
                dst = h5w.create_dataset(trg + "/color", data=ion.color.typed_value)
                dst = h5w.create_dataset(trg + "/volume", dtype=np.float32,
                                         data=ion.volume.typed_value)
                dst.attrs['unit'] = 'nm^3'
                dst = h5w.create_dataset(trg + "/isotope_vector", dtype=np.uint16,
                                         data=ion.isotope_vector.typed_value, 
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(trg + "/nuclid_list", dtype=np.uint16,
                                         data=ion.nuclid_list.typed_value,
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(trg + "/charge", dtype=np.int8,
                                         data=ion.charge.typed_value)
                dst.attrs['unit'] = 'eV'
                dst = h5w.create_dataset(trg + "/name", data=ion.name.typed_value)
                # print(ion.name.typed_value)
                dst = h5w.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                                         data=ion.ranges.typed_value)
                dst.attrs['unit'] = 'Da'
                
                # charge model
                subgrpnm = trg + "/charge_model"
                subgrp = h5w.create_group(subgrpnm)
                subgrp.attrs['NXclass'] = "NXprocess"
                dst = h5w.create_dataset(subgrpnm + "/min_abundance", dtype=np.float64,
                                         data=ion.charge_model['min_abundance'])
                dst = h5w.create_dataset(subgrpnm + "/min_abundance_product", dtype=np.float64,
                                         data=ion.charge_model['min_abundance_product'])
                dst = h5w.create_dataset(subgrpnm + "/min_half_life", dtype=np.float64,
                                         data=ion.charge_model['min_half_life'])
                dst.attrs['unit'] = 's'
                dst = h5w.create_dataset(subgrpnm + "/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                                         data=ion.charge_model['sacrifice_isotopic_uniqueness'])
                dst = h5w.create_dataset(subgrpnm + "/isotope_matrix", dtype=np.uint16,
                                         data=ion.charge_model['isotope_matrix'],
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(subgrpnm + "/charge_vector", dtype=np.int8,
                                         data=ion.charge_model['charge_vector'])
                dst.attrs['unit'] = 'eV'
                dst = h5w.create_dataset(subgrpnm + "/mass_vector", dtype=np.float64,
                                         data=ion.charge_model['mass_vector'])
                dst.attrs['unit'] = 'Da'
                dst = h5w.create_dataset(subgrpnm + "/nat_abun_prod_vector", dtype=np.float64,
                                         data=ion.charge_model['nat_abun_prod_vector'])
                dst = h5w.create_dataset(subgrpnm + "/min_half_life_vector", dtype=np.float64,
                                         data=ion.charge_model['min_half_life_vector'])
                dst.attrs['unit'] = 's'
                       
                i += 1
            h5w.close()

In [ ]:
# H5Web(directory + "/" + filename)
H5Web(filename)

## Matlab FIG

In [ ]:
from ifes_apt_tc_data_modeling.fig.fig_reader import ReadFigTxtFileFormat

In [2]:
fnm = ["R56_01769.rng.fig.txt"]

In [4]:
directory = os.getcwd()
# directory += '/rng/examples_without_provenance/ger_duesseldorf_kuehbach/'
print(directory)

/home/mkuehbach/IFES_APT_TC_RELEASE_GITHUB/ifes_apt_tc_data_modeling/tests/data


In [ ]:
# for filename in os.listdir(directory):
for filename in fnm:
    if filename.lower().endswith(".rng"):
        print(filename)
        if True is True:  # if filename in []:
            # a = ReadRngFileFormat(directory + "/" + filename)
            a = ReadRngFileFormat(filename)
            # a.rng['molecular_ions']
            
            # h5w = h5py.File(directory + "/" + filename + ".nxs", 'w')
            h5w = h5py.File(filename + ".nxs", 'w')
            h5w.create_group("/entry")
            i = 1
            for ion in a.rng['molecular_ions']:
                trg = "/entry/ion" + str(i)
                grp = h5w.create_group(trg)
                grp.attrs['NXclass'] = 'NXion'
                dst = h5w.create_dataset(trg + "/comment", data=ion.comment.typed_value)
                dst = h5w.create_dataset(trg + "/color", data=ion.color.typed_value)
                dst = h5w.create_dataset(trg + "/volume", dtype=np.float32,
                                         data=ion.volume.typed_value)
                dst.attrs['unit'] = 'nm^3'
                dst = h5w.create_dataset(trg + "/isotope_vector", dtype=np.uint16,
                                         data=ion.isotope_vector.typed_value, 
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(trg + "/nuclid_list", dtype=np.uint16,
                                         data=ion.nuclid_list.typed_value,
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(trg + "/charge", dtype=np.int8,
                                         data=ion.charge.typed_value)
                dst.attrs['unit'] = 'eV'
                dst = h5w.create_dataset(trg + "/name", data=ion.name.typed_value)
                # print(ion.name.typed_value)
                dst = h5w.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                                         data=ion.ranges.typed_value)
                dst.attrs['unit'] = 'Da'
                
                # charge model
                subgrpnm = trg + "/charge_model"
                subgrp = h5w.create_group(subgrpnm)
                subgrp.attrs['NXclass'] = "NXprocess"
                dst = h5w.create_dataset(subgrpnm + "/min_abundance", dtype=np.float64,
                                         data=ion.charge_model['min_abundance'])
                dst = h5w.create_dataset(subgrpnm + "/min_abundance_product", dtype=np.float64,
                                         data=ion.charge_model['min_abundance_product'])
                dst = h5w.create_dataset(subgrpnm + "/min_half_life", dtype=np.float64,
                                         data=ion.charge_model['min_half_life'])
                dst.attrs['unit'] = 's'
                dst = h5w.create_dataset(subgrpnm + "/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                                         data=ion.charge_model['sacrifice_isotopic_uniqueness'])
                dst = h5w.create_dataset(subgrpnm + "/isotope_matrix", dtype=np.uint16,
                                         data=ion.charge_model['isotope_matrix'],
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(subgrpnm + "/charge_vector", dtype=np.int8,
                                         data=ion.charge_model['charge_vector'])
                dst.attrs['unit'] = 'eV'
                dst = h5w.create_dataset(subgrpnm + "/mass_vector", dtype=np.float64,
                                         data=ion.charge_model['mass_vector'])
                dst.attrs['unit'] = 'Da'
                dst = h5w.create_dataset(subgrpnm + "/nat_abun_prod_vector", dtype=np.float64,
                                         data=ion.charge_model['nat_abun_prod_vector'])
                dst = h5w.create_dataset(subgrpnm + "/min_half_life_vector", dtype=np.float64,
                                         data=ion.charge_model['min_half_life_vector'])
                dst.attrs['unit'] = 's'
                       
                i += 1
            h5w.close()

In [ ]:
# H5Web(directory + "/" + filename)
H5Web(filename)

## POS

In [ ]:
from ifes_apt_tc_data_modeling.pos.pos_reader import ReadPosFileFormat

In [ ]:
fnm = []

In [ ]:
for filename in fnm:
    pos = ReadPosFileFormat(filename)
    print(pos.filename)
    print(pos.filesize)
    xyz = pos.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = pos.get_mass_to_charge()
    print(np.shape(mq.typed_value))

## ePOS

In [ ]:
from ifes_apt_tc_data_modeling.epos.epos_reader import ReadEposFileFormat

In [ ]:
fnm = []

In [ ]:
for filename in fnm:
    epos = ReadEposFileFormat(filename)
    print(epos.filename)
    print(epos.filesize)
    xyz = epos.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = epos.get_mass_to_charge()
    print(np.shape(mq.typed_value))
    raw_tof = epos.get_raw_time_of_flight()
    print(np.shape(raw_tof.typed_value))
    st_v = epos.get_standing_voltage()
    print(np.shape(st_v.typed_value))
    p_v = epos.get_pulse_voltage()
    print(np.shape(p_v.typed_value))
    det_xy = epos.get_hit_positions()
    print(np.shape(det_xy.typed_value))
    n_p = epos.get_number_of_pulses()
    print(np.shape(n_p.typed_value))
    h_p = epos.get_ions_per_pulse()
    print(np.shape(h_p.typed_value))

## APT

In [ ]:
from ifes_apt_tc_data_modeling.apt.apt6_reader import ReadAptFileFormat

In [ ]:
fnm = []

In [ ]:
for filename in fnm:
    apt = ReadAptFileFormat(filename)
    apt.get_metadata_table()
    # xyz = apt.get_reconstructed_positions()
    # print(np.shape(xyz.typed_value))
    # mq = apt.get_mass_to_charge_state_ratios()
    # print(np.shape(mq.typed_value))

***
Markus Kühbach 2023/03/20